# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.


# Solution
First let's try to do the request on the https://www.topuniversities.com/university-rankings/world-university-rankings/2018 url just to see what we get.

In [1]:
import requests
from bs4 import BeautifulSoup
r = requests.get('https://www.topuniversities.com/university-rankings/world-university-rankings/2018')
soup = BeautifulSoup(r.text, 'html.parser')
soup

<!DOCTYPE html>

<html dir="ltr" version="XHTML+RDFa 1.0" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:article="http://ogp.me/ns/article#" xmlns:book="http://ogp.me/ns/book#" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/ns/product#" xmlns:profile="http://ogp.me/ns/profile#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:schema="http://schema.org/" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:video="http://ogp.me/ns/video#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
<head profile="http://www.w3.org/1999/xhtml/vocab">
<meta content="unsafe-url" name="referrer"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"UwUCVVVTGwIAV1VXBQkP"}

If we do Ctrl+F and try to look for "Imperial College London" for example, we don't get anything in the Beautifoul soup HTML even if the string is clearly on the webpage. This is because the data is not loaded at the beginning when the HTML request is answered. It is loaded later and the ranking data is in a separate file. Using Google Chrome Interceptor we figured out the file is at https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051_indicators.txt. So this is the webpage we will scrape instead of the ranking webpage:

In [2]:
from html import parser
import os

base_url = 'https://www.topuniversities.com/'

# create a subclass and override the handler methods
class QSHTMLParser(parser.HTMLParser):
    def handle_starttag(self, tag, attrs):
        if not hasattr(self, 'array_hrefs'):
            self.array_hrefs = []
        if tag == 'a':
            self.last_tag_url = os.path.join(base_url, attrs[0][1][1:])
            
    def handle_data(self, data):
        self.array_hrefs.append({"url": self.last_tag_url, "name": data})

In [7]:
import json

request_ranking_text_file = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051_indicators.txt')
json_ranking = json.loads(request_ranking_text_file.text)
ranking_data = json_ranking['data'][:50]
parser = QSHTMLParser()
for university in ranking_data:
    parser.feed(university['uni'])
    
basic_data = parser.array_hrefs
for i, university in enumerate(ranking_data):
    curr_uni = basic_data[i]
    curr_uni['region'] = university['region']
    curr_uni['location'] = university['location']

Now we have the URLs of the fifty first universities:

In [8]:
basic_data

[{'location': 'United States',
  'name': 'Massachusetts Institute of Technology (MIT) ',
  'region': 'North America',
  'url': 'https://www.topuniversities.com/universities/massachusetts-institute-technology-mit'},
 {'location': 'United States',
  'name': 'Stanford University',
  'region': 'North America',
  'url': 'https://www.topuniversities.com/universities/stanford-university'},
 {'location': 'United States',
  'name': 'Harvard University',
  'region': 'North America',
  'url': 'https://www.topuniversities.com/universities/harvard-university'},
 {'location': 'United States',
  'name': 'California Institute of Technology (Caltech)',
  'region': 'North America',
  'url': 'https://www.topuniversities.com/universities/california-institute-technology-caltech'},
 {'location': 'United Kingdom',
  'name': 'University of Cambridge',
  'region': 'Europe',
  'url': 'https://www.topuniversities.com/universities/university-cambridge'},
 {'location': 'United Kingdom',
  'name': 'University of Ox

Here we have the URLs for the details pages for the 50 first universities. Let's take a closer look et the EPFL detail page:

In [9]:
import re

url_epfl = basic_data[11]['url']
request_epfl = requests.get(url_epfl)
soup_epfl = BeautifulSoup(request_epfl.text, 'html.parser')
div_details = soup_epfl.find_all('div', {"class": "view-academic-data-profile"})[0]
div_details.find_all('div', {"class": "number"})
test_det = div_details.find_all('div', {"class": "number"})
student_values = ["staff_total", "staff_international", "students_total", "students_international"]
number_students = {}
for i in range(0, 4):
    number = int(re.sub('[,]', '', test_det[i].get_text()))
    number_students[student_values[i]] = number
number_students

{'staff_international': 1300,
 'staff_total': 1695,
 'students_international': 5896,
 'students_total': 10343}

That's it, the code is maybe a little messy but with some browser inspection we can figure out where the things we need on the EPFL webpage are.
Now let us generalize this to all the 50 first universities:

In [12]:
import re

number_students_per_fac = {}
for i, data in enumerate(basic_data):
    url = data['url']
    print("Processing ", i, ": ", url)
    request_uni = requests.get(url)
    soup_uni = BeautifulSoup(request_uni.text, 'html.parser')
    div_details = soup_uni.find_all('div', {"class": "view-academic-data-profile"})[0]
    test_det = div_details.find_all('div', {"class": "number"})
    student_values = ["staff_total", "staff_international", "students_total", "students_international"]
    number_students = {}
    for j in range(0, 4):
        number = int(re.sub('[,]', '', test_det[j].get_text()))
        basic_data[i][student_values[j]] = number
    number_students_per_fac[url] = number_students

Processing  0 :  https://www.topuniversities.com/universities/massachusetts-institute-technology-mit
Processing  1 :  https://www.topuniversities.com/universities/stanford-university
Processing  2 :  https://www.topuniversities.com/universities/harvard-university
Processing  3 :  https://www.topuniversities.com/universities/california-institute-technology-caltech
Processing  4 :  https://www.topuniversities.com/universities/university-cambridge
Processing  5 :  https://www.topuniversities.com/universities/university-oxford
Processing  6 :  https://www.topuniversities.com/universities/ucl-university-college-london
Processing  7 :  https://www.topuniversities.com/universities/imperial-college-london
Processing  8 :  https://www.topuniversities.com/universities/university-chicago
Processing  9 :  https://www.topuniversities.com/universities/eth-zurich-swiss-federal-institute-technology
Processing  10 :  https://www.topuniversities.com/universities/nanyang-technological-university-singapor

In [13]:
basic_data

[{'location': 'United States',
  'name': 'Massachusetts Institute of Technology (MIT) ',
  'region': 'North America',
  'staff_international': 1679,
  'staff_total': 2982,
  'students_international': 3717,
  'students_total': 11067,
  'url': 'https://www.topuniversities.com/universities/massachusetts-institute-technology-mit'},
 {'location': 'United States',
  'name': 'Stanford University',
  'region': 'North America',
  'staff_international': 2042,
  'staff_total': 4285,
  'students_international': 3611,
  'students_total': 15878,
  'url': 'https://www.topuniversities.com/universities/stanford-university'},
 {'location': 'United States',
  'name': 'Harvard University',
  'region': 'North America',
  'staff_international': 1311,
  'staff_total': 4350,
  'students_international': 5266,
  'students_total': 22429,
  'url': 'https://www.topuniversities.com/universities/harvard-university'},
 {'location': 'United States',
  'name': 'California Institute of Technology (Caltech)',
  'region':

That's it, we have all the data for the number of students. We can convince ourselves that this is right by taking a university at random and visiting the corresponding webpage (if we take more of them the probability that our results our wrong decrease exponentially).

In [15]:
import pandas as pd
pd.DataFrame.from_dict(basic_data)

,location,name,region,staff_international,staff_total,students_international,students_total,url
0,United States,Massachusetts Institute of Technology (MIT),North America,1679,2982,3717,11067,https://www.topuniversities.com/universities/m...
1,United States,Stanford University,North America,2042,4285,3611,15878,https://www.topuniversities.com/universities/s...
2,United States,Harvard University,North America,1311,4350,5266,22429,https://www.topuniversities.com/universities/h...
3,United States,California Institute of Technology (Caltech),North America,350,953,647,2255,https://www.topuniversities.com/universities/c...
4,United Kingdom,University of Cambridge,Europe,2278,5490,6699,18770,https://www.topuniversities.com/universities/u...
5,United Kingdom,University of Oxford,Europe,2964,6750,7353,19720,https://www.topuniversities.com/universities/u...
6,United Kingdom,UCL (University College London),Europe,2554,6345,14854,31080,https://www.topuniversities.com/universities/u...
7,United Kingdom,Imperial College London,Europe,2071,3930,8746,16090,https://www.topuniversities.com/universities/i...
8,United States,University of Chicago,North America,635,2449,3379,13557,https://www.topuniversities.com/universities/u...
9,Switzerland,ETH Zurich - Swiss Federal Institute of Techno...,Europe,1886,2477,7563,19815,https://www.topuniversities.com/universities/e...
